In [ ]:
import pyvisa
import pyvisa as visa
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib
import time

# 波形参数
for amplitude in range(10,21,1):
    print(f'amplitude:{amplitude}')

    Group='A'

    # 波形参数
    frequency = 50                   # 频率: kHz
    burst_cycles = 100               # 每次突发周期中的波形周期数
    burst_period = 0.1               # 突发周期：0.1 秒（100 ms）
    sampling_rate = 1e6              # 采样率：1 MHz
    burst_duration = burst_cycles / frequency  # 单次突发信号持续时间

    # 打开信号发生器
    rm = visa.ResourceManager()
    inst = rm.open_resource('TCPIP::192.168.1.100::INSTR')

    # 获取设备信息
    inst.write("*IDN?")
    print("Connected device:", inst.read())

    # 配置脉冲信号参数
    inst.write(":SOURce1:FUNCtion SINusoid")        # 设置信号类型为正弦波
    inst.write(f":SOURce1:FREQuency {frequency}KHz")          # 设置脉冲频率为 50 kHz
    inst.write(f":SOURce1:VOLTage:AMPLitude {amplitude}VPP")  # 设置脉冲幅度为 10 Vpp

    # 配置突发模式
    inst.write(":SOURce1:BURSt:MODE INTernal")      # 使用内部触发模式
    inst.write(f":SOURce1:BURSt:NCYCles {burst_cycles}")        # 每次突发信号包含 100 个脉冲周期
    inst.write(f":SOURce1:BURSt:INT:PERiod {burst_period}")     # 设置突发周期为 100 ms（0.1 秒）
    inst.write(":SOURce1:BURSt:STATe ON")           # 启用突发模式

    # 打开输出
    inst.write(":OUTPut1:STATe ON")

    ###################   示波器部分  ##########################################

    time_interval = 500e-6  #示波器横坐标时间间隔,单位s
    voltage =3    #电压范围,单位V，python图像显示范围
    x_left = -time_interval*0.5       #横坐标起点
    x_right = time_interval*10         #横坐标终点
    y_up = voltage*amplitude*1e-1     #纵坐标上限
    y_down = -voltage*amplitude*1e-1  #纵坐标下限
    y_scale=1000     #垂直刻度，单位mV，示波器纵轴间隔
    x_position=2e-3     #设置触发位置，单位s
    time_pause=10    #暂停时间，单位S
    Average=32  #平均次数

    # 确保使用正确的图形后
    matplotlib.use('TkAgg')

    # 打开示波器
    # 创建ResourceManager对象
    rm_scope = pyvisa.ResourceManager()

    # 连接示波器（请替换为你的实际设备地址）
    scope_address = 'TCPIP::192.168.1.103::INSTR'  # 地址
    scope = rm_scope.open_resource(scope_address)

    # 获取示波器ID信息，确认连接成功
    idn = scope.query('*IDN?')
    scope.write(':RUN')

    trigger_level = amplitude*3E-3  # 设定触发电平单位S
    scope.write(f':TRIGger:SHOLd:DLEVel {trigger_level}')

    # 配置示波器参数（可根据需求修改）
    scope.write(':CHANNEL1:DISPLAY ON')                     # 打开通道1
    scope.write(':CHANNEL1:VOLTAGE:OFFSET 0')               # 设置电压偏移
    scope.write(f':CHANNEL1:VOLTAGE:SCALE {voltage}')       # 设置电压间隔
    scope.write(f':TIMEBASE:SCALE {time_interval}')         # 设置时间间隔
    scope.write(f':CHANnel1:SCALe {y_scale}mv')             # 设置y轴刻度
    scope.write(f':TIMebase:MAIN:OFFSet {x_position}')      #设置触发位置，单位s
    scope.write(':ACQuire:TYPE AVERages')            #平均模式
    scope.write(f':ACQuire:AVERages {Average}')     #平均模式32次

    # 获取波形数据的函数

    # 获取波形数据的函数
    def get_waveform():
        # 设置读取数据点      
        scope.write(':WAVEFORM:FORMAT ASCII')         
        scope.write(':WAVEFORM:SOURCE CHANNEL1')      

        # 读取波形数据
        data = scope.query(':WAVEFORM:DATA?')

        # 去除前缀并清洗数据
        if data.startswith('#'):
            header_length = int(data[1])  # 获取长度标识符
            offset = 2 + header_length     # 计算数据起始位置
            data = data[offset:]           # 保留有效波形数据部分

        # 将数据转换为浮动类型
        data_list = [item for item in data.split(',') if item.strip()]  # 清理空值
        waveform_data = np.array(data_list, dtype=float)

        # 获取X轴的时间增量和起始时间
        x_increment = float(scope.query(':WAVEFORM:XINCREMENT?'))  # 时间增量
        x_origin = float(scope.query(':WAVEFORM:XORIGIN?'))        # 起始时间
        x_values = x_origin + np.arange(len(waveform_data)) * x_increment  # 计算X轴数据
        return x_values, waveform_data

    # 创建图形
    fig, ax = plt.subplots(figsize=(10, 6))
    line, = ax.plot([], [], lw=2)

    # 设置图形的标题和标签
    ax.set_title(f"Real-Time Oscilloscope Waveform for {amplitude}VPP")
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("Voltage (V)")
    ax.grid(True)

    # 设置横坐标和纵坐标范围
    ax.set_xlim([x_left, x_right])  #设置时间范围
    ax.set_ylim([y_down, y_up])    # 设置电压范围

    # 初始化图形（空数据）
    def init():
        line.set_data([], [])
        return line,

    # 更新图形的数据
    def update(frame):
        x_values, waveform_data = get_waveform()
        line.set_data(x_values, waveform_data)
        return line,

    # 创建动画
    ani = animation.FuncAnimation(fig, update, frames=100, init_func=init, blit=True, interval=50)

    # 显示图形
    plt.show()

    # scope.write(":TRIGger:SWEep SINGle")  #设置触发模式为“单次”       
    # scope.write(f':ACQuire:MDEPth {Save_depth}K')  
    # file_path =(f'D:\\{Group}{amplitude}.csv')  # 硬盘的路径                     
    # scope.write(f':SAVE:CSV {file_path}')                                        
    # time.sleep(time_pause)  # 暂停,留出时间存储数据

    scope.write(":TRIGger:SWEep SINGle")  #设置触发模式为“单次” 
    Save_depth=100000
    Collection_depth=Save_depth/1
    scope.write(f':ACQuire:MDEPth {Save_depth}')  
    time.sleep(1)

    rm = pyvisa.ResourceManager()
    scope = rm.open_resource('TCPIP::192.168.1.103::INSTR')  # 替换为实际设备地址
    scope.timeout = 50000  # 设置超时时间,单位ms

    # 设置波形参数
    scope.write(':WAV:SOUR CHAN1')  # 选择通道1
    scope.write(':WAV:MODE MAXimum')  # 设置波形模式为普通模式
    scope.write(':WAV:FORM BYTE')  # 设置波形格式为字节
    scope.write(f':WAVeform:POINts {Collection_depth}')  # 设置采样点数

    # 获取波形参数
    x_increment = float(scope.query(':WAV:XINC?'))  # 时间分辨率
    x_origin = float(scope.query(':WAV:XOR?'))  # 时间起点
    y_increment = float(scope.query(':WAV:YINC?'))  # 电压分辨率
    y_origin = float(scope.query(':WAV:YOR?'))  # 电压起点
    y_reference = float(scope.query(':WAV:YREF?'))  # 电压参考点

    # 读取波形数据（原始值为无符号字节）
    raw_data = scope.query_binary_values(':WAV:DATA?', datatype='B', container=np.array)
    scope.write(":TRIGger:SWEep SINGle")  #设置触发模式为“单次” 
    time.sleep(1)

    # 转换波形数据为物理值
    voltages = (raw_data - y_reference) * y_increment + y_origin
    times = np.arange(0, len(voltages)) * x_increment + x_origin

    print(f'length:{len(voltages)}')

    # 检查转换后的最小值和最大值
    print(f"Voltage range: {voltages.min()} V to {voltages.max()} V")

    # 保存数据到 CSV 文件
    output_file = (f'C:\\Users\\Administrator\\Desktop\\RIGOL\\Demo_python\\探头浸水_园柱混凝土12.5\\{Group}{amplitude}.csv')
    np.savetxt(output_file, np.column_stack((times, voltages)), delimiter=',', header='Time(s),Voltage(V)', comments='')
    print(f"波形数据已保存到 {output_file}")